In [9]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [13]:
##Paths
train_aud = '/Users/tacha/iu_research/speech_recognition/asr_workshop/data/target-segments/ru/clips/'
train_df = '/Users/tacha/iu_research/speech_recognition/asr_workshop/data/target-segments/ru/train.tsv'

**Transform functions**

Transform functions are the functions that are dealing with necessary input transformations e.g. feature extraction. They are feeded directly in the data loader. It helps to speed up data manipulation in contrast to reading all the file from the hard drive.

In [203]:
def find_maxlen(path, train_df):
    fnames = pd.read_csv(train_df, sep='\t')['path']
    maxlen = 0
    for n in tqdm(fnames):
        waveform, sample_rate = torchaudio.load(os.path.join(path, n))
        mfcc = torchaudio.transforms.MFCC()(waveform)
        size = mfcc.shape[2]
        if size > maxlen:
            maxlen = size
    print("Maxlen:", maxlen)


def extract_feats(path, maxlen=1083):
    '''
    Reads and processes one file at a time.
    Args:
        path: path to the file
        maxlen: maximum length of the spectrogram for padding
    '''
    waveform, sample_rate = torchaudio.load(path)
    #Calculate MFCC
    mfcc = torchaudio.transforms.MFCC()(waveform)
    #Calculate delta and double-delta
    deltas = torchaudio.transforms.ComputeDeltas()(mfcc)
    ddeltas = torchaudio.transforms.ComputeDeltas()(deltas)
    res = torch.cat((mfcc, deltas, ddeltas), dim=1).squeeze(0)
    #Normalize rows
    s = torch.sum(res, dim=1, keepdim=True)
    norm = torch.div(res, s)
    mask = torch.ones(norm.shape[0], norm.shape[1])
    padded_norm = nn.functional.pad(norm, pad=(0, maxlen-norm.shape[1], 0, 0), 
                                          mode="constant",value=0)
    padded_mask = nn.functional.pad(mask, pad=(0, maxlen-mask.shape[1], 0, 0), 
                                          mode="constant",value=0)
    return padded_norm, padded_mask

def alphabet_enc(csv_path):
    char2ind = {}
    sents = pd.read_csv(csv_path, sep='\t')['sentence']
    chars = list(set([char for sent in sents for char in sent]))
    for i in range(len(chars)):
        char2ind[chars[i]] = i
    char2ind["<eos>"] = len(chars)+1 
    return char2ind

In [191]:
class TrainData(data.Dataset):
    def __init__(self, csv_path, aud_path, transform):
        self.df = pd.read_csv(csv_path, sep='\t')
        self.aud_path = aud_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        fname = os.path.join(self.aud_path, self.df['path'][idx])
        transcript = self.df['sentence'][idx].lower()

        feat, mask = self.transform(fname)

        sample = {'aud':feat, 'trans': transcript, 'mask':mask}
        return sample
    
def weights(m):
    '''
    Intialize random weights
    '''
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data,0.1)

**Proposed Architechture**

Attention-based Sequence-to-Sequence model:

![](/img/arch.png)

In [219]:
class Encoder(nn.Module):
    def __init__(self, batch_size):
        super().__init__()
        self.input_layer = nn.Linear(120, 512)
        self.blstm = nn.LSTM(input_size=512, 
                             hidden_size=256, 
                             num_layers=3, 
                             bidirectional=True)
        self.h0 = torch.zeros(3*2, batch_size, 256)
        self.c0 = torch.zeros(3*2, batch_size, 256)
        
    def forward(self, x):
        #Pass through the first linear layer
        outputs=[]
        for i in range(x.shape[2]):
            feature = x[:,:,i]
            out = self.input_layer(feature)
            out = torch.nn.LeakyReLU()(out)
            outputs.append(out)
        outputs = torch.stack(outputs)
        #Pass through LSTM layers
        output, (hn, cn) = self.blstm(outputs, (self.h0, self.c0))
        return output, (hn, cn)
    
    
class Decoder(nn.Module):
    def __init__(self, batch_size, char2ind):
        super().__init__()
        self.char2ind = char2ind
        self.embed_layer = nn.Linear(512, 512)
        self.blstm = nn.LSTM(input_size=512, 
                             hidden_size=512, 
                             num_layers=1)
        self.h0 = torch.zeros(1, batch_size, 512)
        self.c0 = torch.zeros(1, batch_size, 512)
        self.y0 = torch.zeros(1, 512)
        

In [209]:
char2ind = alphabet_enc(train_df)

In [210]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(32)
encoder = encoder.to(device)
encoder.apply(weights)

cv_dataset = TrainData(train_df, train_aud, extract_feats)
loader = data.DataLoader(cv_dataset, batch_size=32, shuffle=True)

In [211]:
decoder = Decoder(32, char2ind)

In [189]:
for batch in loader:
    x = batch['aud'].to(device)
    out, (h, c) = encoder(x)
    print(out)

tensor([[[-0.0047,  0.0058, -0.0140,  ..., -0.0589,  0.0014,  0.0187],
         [-0.0050,  0.0068, -0.0076,  ..., -0.0439, -0.0054, -0.0002],
         [-0.0084,  0.0029, -0.0129,  ..., -0.0730, -0.0168,  0.0143],
         ...,
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [-0.0062,  0.0038, -0.0100,  ..., -0.0450, -0.0003,  0.0044],
         [-0.0039,  0.0029, -0.0215,  ..., -0.0810,  0.0007,  0.0024]],

        [[-0.0028,  0.0078, -0.0233,  ..., -0.0677,  0.0016,  0.0210],
         [-0.0033,  0.0103, -0.0124,  ..., -0.0440, -0.0058,  0.0022],
         [-0.0099,  0.0033, -0.0224,  ..., -0.0785, -0.0173,  0.0101],
         ...,
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [-0.0061,  0.0037, -0.0170,  ..., -0.0461,  0.0014,  0.0063],
         [-0.0030,  0.0030, -0.0335,  ..., -0.0823,  0.0051, -0.0011]],

        [[-0.0005,  0.0087, -0.0300,  ..., -0.0797,  0.0032,  0.0190],
         [-0.0003,  0.0120, -0.0156,  ..., -0

KeyboardInterrupt: 